In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'c:\\SSAFY\\S12P21A504\\Data\\globe\\co2'

In [3]:
file_path = './txt/co2/daily/co2_alt4001_surface-insitu_20_9999-9999_daily.txt'

In [4]:
with open(file_path) as f:
  for line in f.readlines():
    print(line)

# header_lines : 220

# Data_Set_Name : co2_alt4001_surface-insitu_20_9999-9999_daily

# Data_Set_Version : 0020-4001-1001-01-01-9999_2025-01-01-0020

# Data_Set_Fair_Use : For Scientific purposes, access to these data is unlimited and provided without charge. By their use you accept that an offer of co-authorship will be made through personal contact with the contributors whenever substantial use is made of their data. In all cases, an acknowledgement must be made to the contributors and to the data centre when these data are used within a publication.

# Data_Set_DOI : 

# Data_Set_Citation_Format : 

#

# GLOBAL ATTRIBUTES

#

# site_name : Alert

# site_country/territory : Canada

# site_wmo_region : REGION IV (North and Central America)

# site_gaw_id : ALT

# site_gaw_type : GAW Global

# site_wigos_id : 0-20008-0-ALT

# site_wdcgg_id : ALT4001

# site_address1 : 

# site_address2 : 

# site_address3 : 

# site_latitude : 82.4991455078

# site_longitude : -62.3415260315

# site_e

In [5]:
total = 0
for (root, dirs, files) in os.walk('./'):
  for file in files:
    file_path = os.path.join(root, file)
    if 'txt' in file_path and 'WDCGG' not in file_path:
      with open(file_path) as f:
        line_count = len(f.readlines())
        print(f"{line_count}\t{file}")
        total += line_count
print(total)

12495	co2_alt4001_surface-insitu_20_9999-9999_daily.txt
17009	co2_cmn6042_surface-insitu_29_9999-9999_daily.txt
11643	co2_mnm2029_surface-insitu_1_9999-9999_daily.txt
18867	co2_ssl6027_surface-insitu_71_9999-9999_daily.txt
3006	co2_bmw4030_surface-flask_2_3001-9999_event.txt
1631	co2_cfa5010_surface-flask_16_9999-9999_event.txt
1123	co2_cpt1009_surface-flask_2_3001-9999_event.txt
834	co2_cri2036_surface-flask_16_9999-9999_event.txt
2710	co2_crz1011_surface-flask_2_3001-9999_event.txt
1024	co2_nat3001_surface-flask_2_3001-9999_event.txt
2233	co2_pal6004_surface-flask_2_3001-9999_event.txt
4634	co2_tap2013_surface-flask_2_3001-9999_event.txt
887	co2_tik2002_surface-flask_2_3001-9999_event.txt
1932	co2_ush3007_surface-flask_2_3001-9999_event.txt
3038	co2_uum2004_surface-flask_2_3001-9999_event.txt
3739	co2_wlg2015_surface-flask_2_3001-9999_event.txt
270931	co2_alt4001_surface-insitu_20_9999-9999_hourly.txt
445266	co2_brw4003_surface-insitu_2_3001-9999_hourly.txt
45168	co2_cai1001_surface-

In [6]:
station_csv = pd.read_csv('./WDCGG_metadata_20250404060512.csv')

In [7]:
station_csv.head()
station_csv.columns

Index(['Station Code', 'GAW ID', 'WDCGG ID', 'WIGOS ID', 'Station Name',
       'Latitude', 'Longitude', 'Elevation', 'WMO Region', 'Country/Territory',
       'Country Code', 'Gaw Category', 'Station Type', 'Contributor',
       'Gas Species', 'Platform', 'Sampling Type', 'Buffer Code', 'Buffer',
       'Start', 'End', 'Measurement Status', 'Scientific Aim', 'Unit',
       'Time Zone', 'Scale'],
      dtype='object')

In [8]:
def find_location(station_code):
  station_csv = pd.read_csv('./WDCGG_metadata_20250404060512.csv')
  for orow in station_csv.iterrows():
    _, row = orow
    if station_code == row["WDCGG ID"].lower():
      country = row["Country Code"]
      lat = row["Latitude"]
      lon = row["Longitude"]

      return (country, lat, lon)
  print(station_code)

In [9]:
result_rows = []
for (root, dirs, files) in os.walk('./'):
  for file in files:
    if "co2" not in file or 'txt' not in file:
      continue
    station_code = file.split("_")[1]
    country_code, latitude, longitude = find_location(station_code)
    
    file_path = os.path.join(root, file)
    with open(file_path) as f:
      for line in f.readlines():
        if "#" in line:
          continue
        sp = line.split(" ")
        year = sp[1]
        month = sp[2]
        day = sp[3]
        hour = sp[4]
        co2 = sp[13]

        if float(co2) < 0:
          continue
        
        result_rows.append({
          "country_code": country_code,
          "latitude": latitude,
          "longitude": longitude,
          "co2": float(co2),
          "reference_date": f"{year}-{month}-{day} {hour}:00:00",
          "year": year,
          "month": month,
          "day": day,
          "hour": hour
        })

df = pd.DataFrame(result_rows)    

In [10]:
aggregated_df = df.groupby(['country_code', 'reference_date'], as_index=False).agg({
    'co2': 'mean',
    'latitude': 'first',
    'longitude': 'first',
    'year': 'first',
    'month': 'first',
    'day': 'first',
    'hour': 'first'
})

aggregated_df.to_csv('./co2.csv', index=False)